## Computantional Theory Problems | Nathan Buyrchiyev

In [4]:
import numpy as np

## Problem 1: Binary Words and Operations

In [ ]:
def parity(x, y, z):
    """
    Calculates the bitwise XOR of three 32-bit unsigned integers.

    This function takes three numbers, converts them to 32-bit unsigned
    integers, and then computes their bitwise XOR. This operation is
    equivalent to calculating the parity for each bit position across
    the three numbers.

    Args:
        x: The first integer.
        y: The second integer.
        z: The third integer.

    Returns:
        A 32-bit unsigned integer representing the bitwise XOR of x, y, and z.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    return x ^ y ^ z

In [18]:

print("--- Basic Tests ---")
# 1 (01) ^ 2 (10) = 3 (11). 3 (11) ^ 3 (11) = 0 (00). Expected: 0x0
test_1 = parity(1, 2, 3) 
print(f"Parity(1, 2, 3) = {hex(test_1)}")

# 0 ^ 0 = 0. 0 ^ 0 = 0. Expected: 0x0
test_2 = parity(0, 0, 0)
print(f"Parity(0, 0, 0) = {hex(test_2)}")

print("\n--- Same Value Tests ---")
# 5 ^ 5 = 0. 0 ^ 5 = 5. Expected: 0x5
test_3 = parity(5, 5, 5)
print(f"Parity(5, 5, 5) = {hex(test_3)}")

# The two 10s cancel out. 10 ^ 10 = 0. 0 ^ 20 = 20. Expected: 0x14
test_4 = parity(10, 20, 10)
print(f"Parity(10, 20, 10) = {hex(test_4)}")

print("\n--- Hash-like Value Tests ---")
# Using large integers that resemble hash values
val_1 = 0xdeadbeef
val_2 = 0xbadf00d
val_3 = 0x12345678
test_5 = parity(val_1, val_2, val_3)
print(f"Parity({hex(val_1)}, {hex(val_2)}, {hex(val_3)}) = {hex(test_5)}")

# Using Python's hash() function, mask to 32 bits to avoid OverflowError
h1 = hash("hello") & 0xFFFFFFFF
h2 = hash("world") & 0xFFFFFFFF
h3 = hash("parity") & 0xFFFFFFFF
test_6 = parity(h1, h2, h3)
print(f"Parity(hash('hello'), hash('world'), hash('parity')) = {hex(test_6)}")

print("\n--- Binary Value Tests ---")
# Using binary literals
# 0b101 (5) ^ 0b010 (2) = 0b111 (7). 0b111 (7) ^ 0b111 (7) = 0. Expected: 0x0
test_7 = parity(0b101, 0b010, 0b111)
print(f"Parity(0b101, 0b010, 0b111) = {hex(test_7)}")

# 1 ^ 1 = 0. 0 ^ 0 = 0. Expected: 0x0
test_8 = parity(1, 1, 0)
print(f"Parity(1, 1, 0) = {hex(test_8)}")

--- Basic Tests ---
Parity(1, 2, 3) = 0x0
Parity(0, 0, 0) = 0x0

--- Same Value Tests ---
Parity(5, 5, 5) = 0x5
Parity(10, 20, 10) = 0x14

--- Hash-like Value Tests ---
Parity(0xdeadbeef, 0xbadf00d, 0x12345678) = 0xc734189a
Parity(hash('hello'), hash('world'), hash('parity')) = 0x14311566

--- Binary Value Tests ---
Parity(0b101, 0b010, 0b111) = 0x0
Parity(1, 1, 0) = 0x0


## Problem 2: Fractional Parts of Cube Roots

## Problem 3: Padding


## Problem 4: Hashes


## Problem 5: Passwords


## End